# Homework week9

**author:** Mehmet Can Ay <br>
2023-12-15

In [1]:
## Install the packages if needed
# !pip install -r requirements.txt

## Import

In [2]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine
import neo4j
import numpy as np
import json
from tqdm import tqdm
import datetime

print(datetime.datetime.now())

2023-12-21 14:10:45.455550


## Create a Student Interaction DataFrame

In [3]:
# Read the student data
df_student: pd.DataFrame = pd.read_csv(
    filepath_or_buffer="./data/students.tsv", 
    sep="\t",
    index_col="id"
)

In [4]:
# Create an empty dataframe with column names
df_student_interaction = pd.DataFrame(
    columns=["interaction_type", "student_a_id", "student_b_id"]
)

In [5]:
# Create an interaction between all students
for i in df_student.index:
    for j in df_student.index:

        if i == j:
            continue
        
        df_student_interaction.loc[len(df_student_interaction.index)] = ["IN_SAME_LAB", i, j]

In [6]:
# Change the index according to SQL conventions
df_student_interaction.index += 1
df_student_interaction.index.rename(name="id", inplace=True)

In [7]:
# Save the dataframe as tsv
df_student_interaction.to_csv(
    path_or_buf="./data/student_interaction.tsv", 
    sep="\t"
)

## Create a phpMyAdmin For students.tsv Data

### MySQL

In [8]:
MYSQL_USER = "root"
MYSQL_HOST = "127.0.0.1"
MYSQL_PORT = 3307
MYSQL_ROOT_PASSWORD = "root_passwd"
MYSQL_DATABASE = "plab2_db"

Create a database cursor

In [9]:
conn = pymysql.connect(
    user=MYSQL_USER,
    password=MYSQL_ROOT_PASSWORD,
    host=MYSQL_HOST,
    port=MYSQL_PORT,
    database=MYSQL_DATABASE,
    autocommit=True,
)
cursor = conn.cursor()

Create an SQL Alchemy engine

In [10]:
connection_url = f"mysql+pymysql://{MYSQL_USER}:{MYSQL_ROOT_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}"
engine = create_engine(connection_url)

In [11]:
def drop_table_if_exists(table: str):
    """Drops a table in MySQL database.

    Args:
        table (str): Name of table.
    """
    cursor.execute(f"DROP TABLE IF EXISTS `{table}`")

#### Create Tables

In [12]:
drop_table_if_exists(table="student_interaction")
drop_table_if_exists(table="student")

In [13]:
sql = """CREATE TABLE IF NOT EXISTS `student` (
  `id` bigint,
  `first_name` text,
  `last_name` text,
  `gitlab_name` text,
  `gitlab_path` text,
  PRIMARY KEY `ix_student_id` (`id`)
)"""
cursor.execute(sql)
conn.commit()

In [14]:
sql = """CREATE TABLE IF NOT EXISTS `student_interaction` (
  `id` bigint,
  `interaction_type` VARCHAR(255),
  `student_a_id` bigint,
  `student_b_id` bigint,
  PRIMARY KEY `ix_student_interaction_id` (`id`),
  FOREIGN KEY (`student_a_id`) REFERENCES student(`id`),
  FOREIGN KEY (`student_b_id`) REFERENCES student(`id`)
)"""
cursor.execute(sql)

0

#### Read the data from file and write into database

##### Student

In [15]:
df_student: pd.DataFrame = pd.read_csv(
    filepath_or_buffer="./data/students.tsv",
    sep="\t",
    index_col="id",
)
df_student.head()

,first_name,last_name,gitlab_name,gitlab_path
id,,,,
1,Aaliya,Anwaar,anwaara0,NaN
2,Afrin,Mohammed Roshanara,mohammeda1,NaN
3,Aparna,Krishnan,krishnana0,NaN
4,Atefeh,Habibi,habibia0,NaN
5,Avani,Bhat,bhata0,NaN


In [16]:
df_student.to_sql(name="student", con=engine, if_exists="append")

16

##### Student Interaction

In [17]:
df_student_interaction: pd.DataFrame = pd.read_csv(
    filepath_or_buffer="./data/student_interaction.tsv",
    sep="\t",
    index_col="id",
)
df_student_interaction.head()

,interaction_type,student_a_id,student_b_id
id,,,
1,IN_SAME_LAB,1,2
2,IN_SAME_LAB,1,3
3,IN_SAME_LAB,1,4
4,IN_SAME_LAB,1,5
5,IN_SAME_LAB,1,6


In [18]:
df_student_interaction.to_sql(
    name="student_interaction", 
    con=engine, 
    if_exists="append"
)

240

## Create a Neo4j For Student Interactions Graph

In [19]:
NEO_PASSWORD = "neo4j_passwd"
NEO_URI = "bolt://localhost:7687"
NEO_USER = "neo4j"
NEO_DB = "neo4j"

In [20]:
def get_cypher_props(props: dict) -> str:
    """Convert dictionary to cypher compliant properties as string."""
    props_str: str = ""
    props_array: list[str] = []
    if props:
        for k, v in props.items():
            if (isinstance(v, (str, int, list)) and v) or (
                isinstance(v, float) and not np.isnan(v)
            ):
                cypher_str: str = f"`{k}`: " + json.dumps(v)
                props_array.append(cypher_str)
        if props_array:
            props_str = "{" + ", ".join(props_array) + "}"
    return props_str

In [21]:
neo_driver: neo4j.Driver = neo4j.GraphDatabase.driver(
    uri=NEO_URI,
    auth=(NEO_USER, NEO_PASSWORD),
    database=NEO_DB,
)
neo_session: neo4j.Session = neo_driver.session()

In [22]:
# delete all node and relationships
neo_session.run("MATCH (n) DETACH DELETE n")

In [23]:
# add nodes
cypher_nodes: list[str] = []
df: pd.DataFrame = pd.read_sql_table(table_name="student", con=engine)
for idx, props in tqdm(df.iterrows(), total=df.shape[0]):
    cypher_props: str = get_cypher_props(props=dict(props))
    cypher_nodes.append("(:Student " + cypher_props + ")")

neo_session.run("CREATE " + ",".join(cypher_nodes))

100%|██████████| 16/16 [00:00<00:00, 3994.10it/s]


In [24]:
df: pd.DataFrame = pd.read_sql_table(
    table_name="student_interaction", 
    con=engine
)
for idx, props in tqdm(df.iterrows(), total=df.shape[0]):
    cypher_props: str = get_cypher_props(props=dict(props))
    cypher = f"""MATCH
        (student_a:Student {{id: {props.student_a_id}}}),
        (student_b:Student {{id: {props.student_b_id}}})
        MERGE (student_a)-[r:IN_SAME_LAB {cypher_props}]->(student_b)"""
    neo_session.run(cypher)

  0%|          | 0/240 [00:00<?, ?it/s]

100%|██████████| 240/240 [00:03<00:00, 76.03it/s]


After all this open http://localhost:7474 and log in with user: neo4j and password: neo4j_passwd.